# LinkedIn Scraper

In [151]:
# Scraping
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

# Misc
import time
from random import randint
from pathlib import Path

# Other modules
from secrets import u_name, pwd

#CSV
import pandas as pd
from csv import writer, reader

In [145]:
# Name of the .csv database
database_name = 'linkedin_database_contacts'

# Variable for not exceeding daily number of clicks
clicks = 0

In [146]:
# Create file if it doesn't exist, otherwise do nothing
file = Path(database_name+".csv")
file.touch(exist_ok=True)

In [147]:
# Initialize webdriver
driver = webdriver.Chrome('/usr/local/bin/chromedriver')
driver.get('https://www.linkedin.com/login')

In [152]:
# Login
time.sleep(randint(2,4))

username = driver.find_element_by_xpath('//input[@name="session_key"]')
password = driver.find_element_by_xpath('//input[@name="session_password"]')

username.send_keys(u_name)
password.send_keys(pwd)

time.sleep(randint(2,4))

submit = driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(randint(2,4))

In [153]:
# 'My Network' button
my_network = driver.find_element_by_xpath('//*[@data-link-to="mynetwork"]').click()

In [154]:
# 'Connections' button
my_connections = driver.find_element_by_xpath('//div[@class="mn-community-summary__entity-info"]').click()

In [155]:
# Getting the total number of connections
connections = int(driver.find_element_by_xpath('//header[@class="mn-connections__header"]').text.split()[0])
connections #No. of connections

102

In [158]:
# Getting the number of visible connections (html)
found_connections = len(driver.find_elements_by_xpath('//div[@class="scaffold-finite-scroll__content"]/ul/li[@class="mn-connection-card artdeco-list"]'))
found_connections #No. of found connections

102

In [157]:
# Scrolling down until the number of visible connections in the page found is equal to the total number of connections
while connections > found_connections:
    time.sleep(randint(2,4))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        driver.find_element_by_xpath('//button[@class="artdeco-button artdeco-button--muted artdeco-button--1 artdeco-button--full artdeco-button--secondary ember-view scaffold-finite-scroll__load-button"]').click()
    except:
        pass
    time.sleep(randint(2,4))
    found_connections = len(driver.find_elements_by_xpath('//div[@class="scaffold-finite-scroll__content"]/ul/li[@class="mn-connection-card artdeco-list"]'))

In [166]:
# Converting every connection to a DataFrame object
names = []
descriptions = []

job_titles = []
companies = []
countries = []
emails = []


database = {'Name': names, 'Description': descriptions}

for contact in driver.find_elements_by_class_name('mn-connection-card__details'):
    name = contact.find_element_by_xpath('.//span[@class="mn-connection-card__name t-16 t-black t-bold"]').text
    description = contact.find_element_by_xpath('.//span[@class="mn-connection-card__occupation t-14 t-black--light t-normal"]').text
    
    names.append(name)
    descriptions.append(description)
    
    print(name, description)
    if name == 'John N., MBA':
        contact.find_element_by_xpath('.//span[@class="mn-connection-card__name t-16 t-black t-bold"]').click()
        break
        
    


In [433]:
# Country
try:
    country = driver.find_element_by_xpath('//div[@class="pb2 pv-text-details__left-panel"]/span[@class="text-body-small inline t-black--light break-words"]').text.split(', ')[-1]
except:
    country = None
    
    
# Job-title and Company
for elem in driver.find_elements_by_class_name('mt4'):
    try:
        if elem.text.split()[0] == 'Experience':
            job_title = elem.find_element_by_xpath('*//div[@class="display-flex align-items-center"]').text.split('\n')[0]
            company = elem.find_element_by_xpath('*//span[@class="t-14 t-normal"]').text.split('\n')[0].split(' ·')[0]
            break
    except IndexError:
        job_title, company = None, None

# Email 
driver.find_element_by_id('top-card-text-details-contact-info').click()
time.sleep(randint(2,4))

try:
    email = driver.find_element_by_xpath('//section[@class="pv-contact-info__contact-type ci-email"]/div').text
except:
    email = None

driver.find_element_by_css_selector("[aria-label=Dismiss]").click()

job_title, company, country, email

(None, None, 'Philippines', None)

In [ ]:
# Printing the DataFrame
df = pd.DataFrame.from_dict(database)
df

In [ ]:
# Appending new connections info if not found in the csv file
with open(database_name+".csv", 'r') as read_obj:
    csv_reader = reader(read_obj)

    for row in df.values.tolist():
        if row not in csv_reader:
            print('Appending: {}'.format(row)) #New connection
            with open (database_name+".csv",'a', newline='') as f_object:
                writer_object = writer(f_object)
                writer_object.writerow(row)
                f_object.close()
            
    read_obj.close()

In [435]:
driver.close()

# MISC

In [ ]:
# #APPENDING NEW VALUES TO DF
# nuevo = ['Pancho', 'Ninguna']
# nuevo_series = pd.Series(nuevo, index = df.columns)
# df = df.append(nuevo_series, ignore_index = True)
# df.tail()